# Library import

In [2]:
import os
import subprocess
import time

import yaml
from sklearn.model_selection import GroupKFold

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm
import torch.nn as nn

from pathlib import Path
import json
from collections import defaultdict, Counter
import random
import random
import shutil
from tqdm import tqdm
import zipfile
from collections import Counter

In [3]:
%pip install ultralytics
import ultralytics
ultralytics.checks()
from ultralytics import YOLO

Ultralytics 8.3.57 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6037.6/8062.4 GB disk)


# Path

In [5]:
COCO_JSON_NM = 'COCO_annotations_new.json'
OUT_COCO_JSON_NM = 'mod_COCO_annotations.json'
OUT_IMAGE_FLDR_NM = 'images'
TRAIN_NM = 'train'
VAL_NM = 'val'
TEST_NM = 'test'
LABEL_NM = 'labels'
YAML = 'dataset.yaml'
RANDOM_SEED = 2023

in_dataset_pth = Path('/kaggle/input/our-xview-dataset')
yolo_dataset_pth = Path('/kaggle/input/yolodataset-xview')
out_dataset_pth = Path('/kaggle/working/')
img_fldr = Path(f'/kaggle/input/our-xview-dataset/{OUT_IMAGE_FLDR_NM}')

coco_json_pth = in_dataset_pth / COCO_JSON_NM
new_coco_json_pth = out_dataset_pth / OUT_COCO_JSON_NM

train_img = yolo_dataset_pth / TRAIN_NM / OUT_IMAGE_FLDR_NM
val_img = yolo_dataset_pth / VAL_NM / OUT_IMAGE_FLDR_NM
test_img = yolo_dataset_pth / TEST_NM / OUT_IMAGE_FLDR_NM
train_txt = yolo_dataset_pth / TRAIN_NM / LABEL_NM
val_txt = yolo_dataset_pth / VAL_NM / LABEL_NM 
test_txt = yolo_dataset_pth / TEST_NM / LABEL_NM  
dataset_yaml = yolo_dataset_pth / YAML

In [4]:
# Pulizia dell'output per cartelle specifiche
def clean_output(output_dir):
    if output_dir.exists() and output_dir.is_dir():
        for item in output_dir.iterdir():
            if item.is_dir():
                shutil.rmtree(item)  # Rimuove la sotto-cartella
            else:
                item.unlink()  # Rimuove il file
        print(f"Cartella {output_dir} pulita.")
    else:
        print(f"Cartella {output_dir} non trovata. Nessuna azione necessaria.")

# Pulisce la cartella di output prima di avviare il processo
clean_output(out_dataset_pth)

Cartella /kaggle/working pulita.


# Utility

In [ ]:
def load_json(file_path):
    """
    Carica un file JSON dal percorso specificato.

    :param file_path: Percorso al file JSON da caricare.
    :return: Dati contenuti nel file JSON (come dizionario o lista).
    """
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

# COCO Preprocessing

In [ ]:
def process_custom_coco_json(input_path, output_path):
    """
    Funzione per processare un JSON COCO in formato personalizzato.
    """
    # Leggi il JSON dal file di input
    data = load_json(input_path)

    # Ottieni e correggi il formato delle categorie
    raw_categories = data.get('categories', [])
    categories = []
 
    for category in tqdm(raw_categories, desc="Processing Categories"):
        for id_str, name in category.items():
            try:
                categories.append({"id": int(id_str), "name": name})
            except ValueError:
                print(f"Errore nel parsing della categoria: {category}")
 
    # Preprocessa le annotazioni in un dizionario per immagini
    image_annotations_dict = {}
    for annotation in tqdm(data.get('annotations', []), desc="Building Image Annotations Dictionary"):
        image_id = annotation['image_id']
        if image_id not in image_annotations_dict:
            image_annotations_dict[image_id] = []
        image_annotations_dict[image_id].append(annotation)
 
    # Elenco di annotazioni da mantenere (solo quelle valide)
    valid_annotations = []
    annotations_to_remove = set()
 
    # Controllo dei bounding box
    for annotation in tqdm(data.get('annotations', []), desc="Processing Annotations"):
        
        # Converte il formato del bbox
        if isinstance(annotation['bbox'], str):
            annotation['bbox'] = json.loads(annotation['bbox'])
        
        x, y, width, height = annotation['bbox']
        xmin, xmax = x, x + width
        ymin, ymax = y, y + height
        
        # Verifica che xmin < xmax e ymin < ymax, e che la larghezza e altezza siano sufficienti
        if xmin >= xmax or ymin >= ymax or width <= 10 or height <= 10:
            annotations_to_remove.add(annotation['id'])
        else:
            annotation['bbox'] = [xmin, ymin, xmax, ymax]
            valid_annotations.append(annotation)
 
    # Rimuovi le annotazioni non valide
    data['annotations'] = valid_annotations
 
    # Ordina le categorie per ID
    categories = sorted(categories, key=lambda x: x['id'])
    
    # Aggiorna le categorie nel JSON
    data['categories'] = categories

 
    # Scrivi il JSON modificato nel file di output
    with open(output_path, 'w') as f:
        json.dump(data, f, indent=4)

In [ ]:
process_custom_coco_json(coco_json_pth, new_coco_json_pth)

### Category Check

In [ ]:
def count_bounding_boxes(json_path):
    """
    Conta il numero di bounding box per ogni categoria in un file COCO JSON.

    Args:
        json_path (str): Percorso del file JSON.

    Returns:
        list: Elenco di tuple con ID categoria, nome categoria e numero di bounding box.
    """
    # Carica il file JSON
    coco_data = load_json(json_path)

    # Estrarre i dati principali
    annotations = coco_data.get("annotations", [])
    categories = coco_data.get("categories", [])

    # Mappare id di categoria ai nomi delle categorie
    category_id_to_name = {category["id"]: category["name"] for category in categories}

    # Contare i bounding box per categoria
    bbox_counts = defaultdict(int)
    for annotation in annotations:
        category_id = annotation["category_id"]
        bbox_counts[category_id] += 1

    # Creare un elenco dei risultati
    results = [
        (cat_id, category_id_to_name.get(cat_id, "Unknown"), count)
        for cat_id, count in bbox_counts.items()
    ]
    
    # Ordinare i risultati in ordine decrescente per numero di bounding box
    results.sort(key=lambda x: x[2], reverse=True)
    
    # Stampare i risultati
    for cat_id, category_name, count in results:
        print(f"Categoria ID {cat_id} ('{category_name}'): {count} bounding box")

In [ ]:
count_bounding_boxes(new_coco_json_pth)

# JSON to YOLO

In [ ]:
def convert_json_to_yolo(json_path, images_dir, output_dir, input_dir, train_dir_out, val_dir_out, test_dir_out, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    # Carica il file JSON
    with open(json_path) as f:
        data = json.load(f)

    # Mappa le classi
    class_mapping = {category['id']: category['name'] for category in data['categories']}
    nc = len(class_mapping)  # Numero di classi

    # Crea le cartelle per il dataset
    train_images_dir = os.path.join(output_dir, 'train', 'images')
    val_images_dir = os.path.join(output_dir, 'val', 'images')
    test_images_dir = os.path.join(output_dir, 'test', 'images')

    train_labels_dir = os.path.join(output_dir, 'train', 'labels')
    val_labels_dir = os.path.join(output_dir, 'val', 'labels')
    test_labels_dir = os.path.join(output_dir, 'test', 'labels')
    
    for dir_path in [train_images_dir, val_images_dir, test_images_dir, train_labels_dir, val_labels_dir, test_labels_dir]:
        os.makedirs(dir_path, exist_ok=True)

    # Dividi le immagini in training, validation, and test
    images = data['images']
    random.shuffle(images)
    total_images = len(images)
    
    train_split = int(train_ratio * total_images)
    val_split = int((train_ratio + val_ratio) * total_images)

    train_images = images[:train_split]
    val_images = images[train_split:val_split]
    test_images = images[val_split:]

    # Funzione per copiare le immagini in una cartella specifica
    def copy_images(image_list, target_dir):
        for image in tqdm(image_list, desc=f"Copying images to {target_dir}", unit="image"):
            src_path = os.path.join(images_dir, image['file_name'])
            dst_path = os.path.join(target_dir, image['file_name'])
            shutil.copy(src_path, dst_path)

    # Copia le immagini nelle rispettive cartelle
    copy_images(train_images, train_images_dir)
    copy_images(val_images, val_images_dir)
    copy_images(test_images, test_images_dir)

    # Converte le annotazioni in formato YOLO e salva nei file di testo
    def convert_annotations(image, annotations, target_dir):
        image_id = image['id']
        image_width = image['width']
        image_height = image['height']
        image_name = image['file_name']

        label_file_path = os.path.join(target_dir, f"{image_name.replace('.jpg', '.txt')}")
        label_dir = os.path.dirname(label_file_path)

        # Crea la cartella se non esiste
        os.makedirs(label_dir, exist_ok=True)

        with open(label_file_path, 'w') as label_file:
            for annotation in annotations:
                if annotation['image_id'] == image_id:
                    category_id = annotation['category_id']
                    xmin, ymin, xmax, ymax = annotation['bbox']

                    # Normalizza le coordinate
                    x_center = (xmin + xmax) / 2 / image_width
                    y_center = (ymin + ymax) / 2 / image_height
                    width = (xmax - xmin) / image_width
                    height = (ymax - ymin) / image_height

                    # Scrivi nel file di annotazione
                    label_file.write(f"{category_id} {x_center} {y_center} {width} {height}\n")

    # Converte le annotazioni per ogni immagine e le salva nelle cartelle appropriate
    def process_images(image_list, target_dir, label_target_dir):
        for image in tqdm(image_list, desc=f"Converting annotations", unit="image"):
            annotations = [annotation for annotation in data['annotations'] if annotation['image_id'] == image['id']]
            convert_annotations(image, annotations, label_target_dir)

    # Processa le immagini per training, validation e test
    process_images(train_images, 'train', train_labels_dir)
    process_images(val_images, 'val', val_labels_dir)
    process_images(test_images, 'test', test_labels_dir)
        
    # Crea il file YAML per YOLO
    yaml_content = f"""path: {input_dir}
train: {train_dir_out}  # train images
val: {val_dir_out}  # val images
test: {test_dir_out}  # test images

# Classes
nc: {nc}  # number of classes
names:
"""
    # Aggiungi le classi al file YAML
    for idx, class_name in class_mapping.items():
        yaml_content += f"  {int(idx)}: {class_name}\n"  # Assicurati che l'indice sia numerico senza virgolette

    # Scrivi il file YAML
    with open(os.path.join(output_dir, 'dataset.yaml'), 'w') as yaml_file:
        yaml_file.write(yaml_content.strip())

    print("Conversione e split completati.")

In [ ]:
convert_json_to_yolo(
    json_path=new_coco_json_pth,
    images_dir=img_fldr,
    output_dir=out_dataset_pth,
    input_dir=yolo_dataset_pth,
    train_dir_out=train_img,
    val_dir_out=val_img,
    test_dir_out=test_img,
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1
)

In [ ]:
# Nome del file zip da creare
zip_file_name = "YOLO_dataset.zip"

# Elenco di file e cartelle da includere nello zip
items_to_zip = [
    "test",
    "train",
    "val",
    "dataset.yaml",
]

# Funzione per aggiungere file e cartelle allo zip
def zip_folder(zipf, folder_path, base_folder=""):
    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, base_folder)
            zipf.write(file_path, arcname)

# Creazione dello zip
with zipfile.ZipFile(zip_file_name, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
    for item in items_to_zip:
        if os.path.exists(item):  # Verifica che il file o la cartella esista
            if os.path.isdir(item):  # Se è una cartella, aggiungi tutto il contenuto
                zip_folder(zipf, item, out_dataset_pth)
            else:  # Se è un file, aggiungilo direttamente
                zipf.write(item)
        else:
            print(f"Elemento non trovato: {item}")

## Check Yaml

In [ ]:
def analyze_yolo_dataset(yaml_path):
    # Carica il file YAML
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    # Estraggo le informazioni dal file YAML
    base_path = data['path']
    sets = ['train', 'val', 'test']
    stats = {}
    
    total_images = 0
    total_bboxes = Counter()
    class_names = data['names']

    for dataset_set in sets:
        images_path = os.path.join(base_path, dataset_set)
        labels_path = os.path.join(base_path, dataset_set, 'labels')

        # Controlla se le directory esistono
        if not os.path.exists(images_path) or not os.path.exists(labels_path):
            print(f"Directory per il set '{dataset_set}' non trovata.")
            continue

        # Conta immagini
        image_files = [f for f in os.listdir(images_path) if f.endswith(('.jpg'))]
        num_images = len(image_files)
        total_images += num_images
        
        stats[dataset_set] = {"images": num_images, "bbox": Counter()}

        # Conta bounding box con tqdm
        print(f"Analizzando {dataset_set}...")
        for label_file in tqdm(os.listdir(labels_path), desc=f"Contando bbox in {dataset_set}", unit="file"):
            label_path = os.path.join(labels_path, label_file)
            with open(label_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    class_id = line.split()[0]  # La classe è il primo elemento della riga
                    stats[dataset_set]["bbox"][class_id] += 1
                    total_bboxes[class_id] += 1

    return stats, total_images, total_bboxes, class_names

# Funzione per mostrare i risultati con le percentuali calcolate per set
def print_stats(stats, total_images, total_bboxes, class_names):
    print("\n--- ANALISI COMPLESSIVA ---")
    print(f"Totale immagini: {total_images}")
    print(f"Distribuzione complessiva delle categorie:")
    
    # Ordina le classi in base al numero di bbox, dal più grande al più piccolo
    sorted_bboxes = sorted(total_bboxes.items(), key=lambda x: x[1], reverse=True)
    
    for class_id, count in sorted_bboxes:
        percentage = (count / sum(total_bboxes.values())) * 100
        print(f"  Classe {class_names[int(class_id)]} ({class_id}): {count} bbox ({percentage:.2f}%)")
    
    print("\n--- ANALISI PER SET ---")
    
    total_bboxes_all_sets = 0  # Per calcolare il totale complessivo dei bbox
    for dataset_set, data in stats.items():
        images_percentage = (data['images'] / total_images) * 100 if total_images > 0 else 0
        print(f"\n--- {dataset_set.upper()} ---")
        print(f"Numero di immagini: {data['images']} ({images_percentage:.2f}%)")
        print("Distribuzione categorie:")
        
        # Calcola il totale dei bbox per ogni set
        total_bboxes_in_set = sum(data['bbox'].values())
        total_bboxes_all_sets += total_bboxes_in_set  # Aggiungi al totale complessivo
        
        # Ordina le classi per ogni set in base al numero di bbox
        sorted_set_bboxes = sorted(data['bbox'].items(), key=lambda x: x[1], reverse=True)
        
        for class_id, count in sorted_set_bboxes:
            percentage = (count / total_bboxes_in_set) * 100 if total_bboxes_in_set > 0 else 0
            print(f"  Classe {class_names[int(class_id)]} ({class_id}): {count} bbox ({percentage:.2f}%)")
        
        print("Totale bounding box:", total_bboxes_in_set, f"({(total_bboxes_in_set / sum(total_bboxes.values())) * 100:.2f}%)")

    # Aggiungi la percentuale per il totale complessivo dei bounding box
    print("\n--- TOTALE COMPLESSIVO ---")
    print(f"Totale bounding box: {total_bboxes_all_sets}")

In [ ]:
# Esegui analisi
stats, total_images, total_bboxes, class_names = analyze_yolo_dataset(dataset_yaml)
print_stats(stats, total_images, total_bboxes, class_names)

# YoloV11

## Training

In [8]:
# Pretrained Model
model = YOLO('yolo11n.pt') 

# Configurazione del training
results = model.train(
    data=dataset_yaml,     # Path al file YAML del dataset
    epochs=100,            # Numero di epoche
    batch=16,              # Dimensione del batch
    imgsz=640,             # Dimensione delle immagini (640x640)
    lr0=0.01,              # Learning rate iniziale
    lrf=0.1,               # Fattore di decay del learning rate ((riduce da 0.01 a 0.001))
    save=True,             # Salva i pesi migliori
    #optimizer='AdamW',    
    optimizer = 'SGD',
    momentum=0.9,          # Momentum ottimizzato
    freeze=[0, 1, 2]       # Congela i primi tre livelli (backbone)
    #project='my_model_results'  # si può personalizzare la directory di salvataggio dei pesi
)

Ultralytics 8.3.57 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/kaggle/input/yolodataset-xview/dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=[0, 1, 2], multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_con

train: Scanning /kaggle/input/yolodataset-xview/train/labels... 36712 images, 11847 backgrounds, 0 corrupt: 100%|██████████| 36712/36712 [00:45<00:00, 810.47it/s]

train: WARNING ⚠️ /kaggle/input/yolodataset-xview/train/images/img_1449_960_1280.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /kaggle/input/yolodataset-xview/train/images/img_1832_0_0.jpg: 69 duplicate labels removed
train: WARNING ⚠️ /kaggle/input/yolodataset-xview/train/images/img_1832_0_1600.jpg: 36 duplicate labels removed
train: WARNING ⚠️ /kaggle/input/yolodataset-xview/train/images/img_1832_0_1920.jpg: 44 duplicate labels removed
train: WARNING ⚠️ /kaggle/input/yolodataset-xview/train/images/img_1832_0_2240.jpg: 43 duplicate labels removed
train: WARNING ⚠️ /kaggle/input/yolodataset-xview/train/images/img_1832_0_2560.jpg: 23 duplicate labels removed
train: WARNING ⚠️ /kaggle/input/yolodataset-xview/train/images/img_1832_0_320.jpg: 66 duplicate labels removed
train: WARNING ⚠️ /kaggle/input/yolodataset-xview/train/images/img_1832_0_640.jpg: 50 duplicate labels removed
train: WARNING ⚠️ /kaggle/input/yolodataset-xview/train/images/img_1896_0_0.jpg: 59 duplicate labels removed

train: WARNING ⚠️ Cache directory /kaggle/input/yolodataset-xview/train is not writeable, cache not saved.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/input/yolodataset-xview/val/labels... 4589 images, 1496 backgrounds, 0 corrupt: 100%|██████████| 4589/4589 [00:05<00:00, 828.48it/s]

...

                   all       4589      48075      0.636      0.379      0.396        0.2

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     11/100      4.24G      1.723      1.531      1.509        197        640: 100%|██████████| 2295/2295 [06:14<00:00,  6.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 144/144 [00:35<00:00,  4.09it/s]


                   all       4589      48075      0.617      0.406      0.404      0.204

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     12/100      3.81G      1.718      1.514       1.51        218        640:  15%|█▌        | 348/2295 [00:56<05:17,  6.12it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()



KeyboardInterrupt: 

### Creazione zip 

In [7]:
# Nome del file zip da creare
zip_file_name = "SGD_decay_epoch84.zip"

# Percorso della cartella da includere nello zip
folder_to_zip = "/kaggle/working/runs/detect/train4"  

# Funzione per aggiungere file e cartelle allo zip
def zip_folder(zipf, folder_path, base_folder=""):
    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, base_folder)  # Percorso relativo per preservare la struttura
            zipf.write(file_path, arcname)

# Creazione dello zip
if os.path.exists(folder_to_zip):  # Verifica che la cartella esista
    with zipfile.ZipFile(zip_file_name, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
        zip_folder(zipf, folder_to_zip, base_folder=folder_to_zip)
    print(f"Archivio zip creato: {zip_file_name}")
else:
    print(f"Cartella non trovata: {folder_to_zip}")

Archivio zip creato: SGD_decay_epoch84.zip


### Riprendere addestramento da pesi salvati

In [6]:
# Carica il modello con i pesi precedenti
model = YOLO('/kaggle/input/sgd_decay/pytorch/default/1/last.pt') 

# Configura il training
results = model.train(
    data=dataset_yaml,            # Percorso al dataset YAML
    epochs=100,                    
    batch=16,                     # Dimensione del batch
    imgsz=640,                    # Dimensione delle immagini
    lr0=0.01,                     # Learning rate iniziale
    lrf=0.1,                      # Fattore di decay del learning rate (riduce da 0.01 a 0.001)
    optimizer='SGD',              # Ottimizzatore
    momentum=0.9,                 # Momentum
    freeze=[0,1,2],
    save=True,                    # Salva i nuovi pesi
    resume=True                   # Indica di riprendere l'addestramento
)

Ultralytics 8.3.57 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/input/sgd_decay/pytorch/default/1/last.pt, data=/kaggle/input/yolodataset-xview/dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=/kaggle/input/sgd_decay/pytorch/default/1/last.pt, amp=True, fraction=1.0, profile=False, freeze=[0, 1, 2], multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=

100%|██████████| 755k/755k [00:00<00:00, 17.7MB/s]


TensorBoard: Start with 'tensorboard --logdir runs/detect/train4', view at http://localhost:6006/

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True] 

100%|██████████| 5.35M/5.35M [00:00<00:00, 74.7MB/s]


AMP: checks passed ✅


...

                   all       4589      48075      0.597      0.524      0.539      0.293

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     84/100      6.31G      1.603      1.274      1.406         77        640: 100%|██████████| 2295/2295 [06:01<00:00,  6.35it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 144/144 [00:34<00:00,  4.12it/s]


                   all       4589      48075        0.6      0.523      0.541      0.294

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     85/100      3.85G      1.582      1.242      1.393        146        640:   7%|▋         | 162/2295 [00:25<05:51,  6.07it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
     85/100      3.85G      1.582      1.242      1.393        146        640:   7%|▋         | 162/2295 [00:25<05:38,  6.30it/s]


KeyboardInterrupt: 

## Validazione

In [ ]:
model = YOLO('/kaggle/working/runs/detect/train/weights/best.pt') # carica i pesi salvati (senza si considera il modello dell'ultima epoca)
model.eval()
results = model.val(data=dataset_yaml) 

## Test

In [8]:
model = YOLO('/kaggle/working/runs/detect/train4/weights/best.pt')
results = model.predict(test_img, imgsz=416, conf = 0.25, save = True, project = out_dataset_pth, verbose = False)


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs



KeyboardInterrupt: 

In [ ]:
#visualizza un'immagine di test con box e etichette predette
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Specifica il percorso dell'immagine
path = "predict/img_102_0_2560.jpg"

# Carica l'immagine
img = mpimg.imread(path)

# Mostra l'immagine
plt.imshow(img)
plt.axis('off')  # Rimuove gli assi
plt.show()